# KNN

Do wykonania naszego modelu wykorzystaliśmy algorytm KNN, z racji na swoją prostotę oraz zasadę działania. <br/>
Pozwala on na znajdowanie podobnych rzeczy, z tego to powodu jest dobrym wyborem do sugerowania filmów i seriali. 

# Import Libraries

In [16]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler, OneHotEncoder
from sklearn.neighbors import NearestNeighbors
import numpy as np

pd.options.display.width = 1000

# Read data

In [4]:
df = pd.read_csv('dataset/cleaned_movies.csv')

# Data transforming

Jako `genres` jest listą, w format plików csv nie posiada natywnej reprezentacji listy, musimy zmapować wszystkie `genres` na listy.

In [5]:
df['genres'] = df['genres'].fillna('').apply(lambda x: x.split(', ') if x else [])

# Data encoding

Po przekrztałceniu danych na wymagane dla algorytmu dane musimy utworzyć i pogrupować parametry na podstawie których będą porównywane filmy.

Biblioteka, której użyliśmy, pozwala na wektoryzacje list, enumeracji oraz oczywiście skalarów.

In [ ]:
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(df['genres'])

# One-hot encoding dla języka
ohe = OneHotEncoder(sparse_output=False)
language_encoded = ohe.fit_transform(df[['original_language']])

# Skalowanie runtime, vote_average, vote_count
scaler = StandardScaler()
numeric_features = scaler.fit_transform(df[['runtime', 'vote_average', 'vote_count']])

# Vectors

In [7]:
X = np.hstack([genres_encoded, language_encoded, numeric_features])

# Model

Po utworzeniu stosu parametrów wejściowych to strukturę odpowiedzialną za wykonywanie obliczeń algorytmu KNN.

In [8]:
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(X)

NearestNeighbors(algorithm='brute', metric='cosine')

# Functions

Dodatkowo dla celów użytkowych stowżyliśmy kilka funkcji pomocniczych, ułatwiających wykonywanie zapytań do naszego modelu.

Z racji na zasadę działania algorytmu (przyjmuje 1 punkt) oraz na naszą chęć możliwości podawania listy lubianych filmów i seriali,<br>
zmuszeni zostaliśmy do kilkukrotnego wykonania zapytania do naszego modelu oraz do dodania i uśrednienia otrzymanych wyników w celu uzyskania porządanego wyniku,<br>
czyli list proponowanych następynch filmów i seriali.

In [11]:
def recommend_similar_movies(movie_index, n_recommendations=5):
    distances, indices = knn.kneighbors([X[movie_index]], n_neighbors=n_recommendations + 1)
    recommendations = indices.flatten()[1:]
    return df.iloc[recommendations][['original_title', 'genres', 'vote_average']]

def recommend_based_on_multiple_movies(movie_indices, n_recommendations=5):
    # Średnia wektorów cech dla wybranych filmów
    mean_features = np.mean(X[movie_indices], axis=0).reshape(1, -1)
    distances, indices = knn.kneighbors(mean_features, n_neighbors=n_recommendations + len(movie_indices))
    
    # Usuń z wyników filmy już znane użytkownikowi
    recommendations = []
    for idx in indices.flatten():
        if idx not in movie_indices:
            recommendations.append(idx)
        if len(recommendations) == n_recommendations:
            break
    return df.iloc[recommendations][['original_title', 'genres', 'vote_average']]

def get_movie_index_by_title(title):
    matches = df[df['original_title'].str.lower() == title.lower()]
    if not matches.empty:
        return matches.index[0]
    else:
        raise ValueError(f"Movie '{title}' not found.")

# Test

In [18]:
tests = [
    ["Toy Story", "Jumanji"],
    ["Toy Story", "Jumanji", "Nadja"],
    ["Toy Story", "Jumanji", "Nadja", "Thinner"],
    ["Toy Story", "Jumanji", "Nadja", "Thinner", "The Lion King"],
    ["Toy Story", "Jumanji", "Nadja", "Thinner", "The Lion King", "The Nightmare Before Christmas"],
]

for movie_titles in tests:
    try:
        print(f"=========================================================================================")
        movie_indices = [get_movie_index_by_title(title) for title in movie_titles]
        recommendations = recommend_based_on_multiple_movies(movie_indices, n_recommendations=5)
        print("Rekomendacje na podstawie filmów:", movie_titles)
        print(recommendations)
        print(f"=========================================================================================\n")
    except ValueError as e:
        print(e)

Rekomendacje na podstawie filmów: ['Toy Story', 'Jumanji']
                 original_title                                           genres  vote_average
36078                  Zootopia           [Animation, Adventure, Family, Comedy]           7.7
13710                        Up           [Animation, Comedy, Family, Adventure]           7.8
4168                      Shrek  [Adventure, Animation, Comedy, Family, Fantasy]           7.3
14965  How to Train Your Dragon          [Fantasy, Adventure, Animation, Family]           7.5
19686            Wreck-It Ralph           [Family, Animation, Comedy, Adventure]           7.1

Rekomendacje na podstawie filmów: ['Toy Story', 'Jumanji', 'Nadja']
                 original_title                                           genres  vote_average
4168                      Shrek  [Adventure, Animation, Comedy, Family, Fantasy]           7.3
14965  How to Train Your Dragon          [Fantasy, Adventure, Animation, Family]           7.5
36078            